In [27]:
import os
import gradio as gr
import requests
from typing import List
from openai import OpenAI
from dotenv import load_dotenv
from bs4 import BeautifulSoup

In [7]:
load_dotenv()
api_key = os.getenv("OPENAI_API")
if api_key and api_key[:8] == "sk-proj-":
    print("API KEY IS READY")
else:
    print("Something went wrong")

API KEY IS READY


In [12]:
openai = OpenAI(api_key=api_key)
MODEL = "gpt-4o-mini"

In [11]:
system_prompt = """
You are a helpful assistant please answer to the user's query precisely.
If you don't know the answer just say that I am not familiar with it.
"""

In [13]:
def message_gpt(prompt):
    messages = [
        {"role":"system","content":system_prompt},
        {"role":"user","content":prompt}
    ]
    completion = openai.chat.completions.create(
        model = MODEL,
        messages=messages,
    )
    response = completion.choices[0].message.content
    return response

In [14]:
message_gpt("What is todays date")

"Today's date is October 24, 2023."

In [15]:
def UP_UP(text):
    return text.upper()

In [ ]:
gr.Interface(fn=UP_UP,inputs="textbox",outputs="textbox",allow_flagging="never").launch()
# gr.Interface(fn=UP_UP,inputs="textbox",outputs="textbox",allow_flagging="never").launch(share=True)

d:\MASTERING_LLMs\llmEnv\lib\site-packages\gradio\interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [19]:
view = gr.Interface(
    fn=message_gpt,
    inputs=gr.Textbox(label="Your Message",lines=6),
    outputs=gr.Textbox(label="Response",lines=8),
    allow_flagging="never"
)
view.launch()

d:\MASTERING_LLMs\llmEnv\lib\site-packages\gradio\interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [22]:
view = gr.Interface(
    fn=UP_UP,
    inputs=gr.Textbox(label="Your Message",lines=6),
    outputs=gr.Markdown(label="Response"),
    allow_flagging="never"
)
view.launch()

d:\MASTERING_LLMs\llmEnv\lib\site-packages\gradio\interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [23]:
def stream_output(prompt):
    messages = [
        {"role":"system","content":system_prompt},
        {"role":"user","content":prompt}
    ]
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [26]:
view = gr.Interface(
    fn=stream_output,
    inputs=gr.Textbox(label="Your Message:"),
    outputs=gr.Markdown(label="Response"),
    allow_flagging="never",
)
view.launch()

d:\MASTERING_LLMs\llmEnv\lib\site-packages\gradio\interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [ ]:
class Website:
    def __init__(self,url):
        self.url = url
        response = requests.get(self.url)
        self.body = response.content
        soup = BeautifulSoup(self.body,"html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator='\n',strip=True)
    def get_contents(self):
        return f"Webpage Title: {self.title}\nWebpage Contents:\n{self.text}\n\n"

In [35]:
system_prompt = "You are an assitant that analyzes the contents of a company landing page \
and create a short brochure about the company for customers, investors and recruiters. Respond in Markdown"

In [41]:
def stream_brochure(title,url):
    prompt = f"Please provide a company brochure for {title}, here is their landing page:\n{Website(url).get_contents()}"
    result = stream_output(prompt=prompt)
    for chunk in result:
        yield chunk

In [ ]:
# stream_brochure("Anthropic","https://anthropic.com")

'Please provide a company brochure for Anthropic, here is their landing page:\nWebpage Title: Home \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nTeam plan\nEnterprise plan\nEducation plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certification\nLearn\nLearning resources\nCustomer stories\nEngineering at Anthropic\nAnthropic Academy\nCompany\nAbout\nCareers\nEvents\nEnginee

In [42]:
view = gr.Interface(
    fn=stream_brochure,
    inputs= [gr.Textbox(label="Company name: "),
             gr.Textbox(label="Landing Page URL: "),
             ],
    outputs=gr.Markdown(label="Brochure:"),
    allow_flagging="never",
)
view.launch()

d:\MASTERING_LLMs\llmEnv\lib\site-packages\gradio\interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


'Webpage Title: Hugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnanonets/Nanonets-OCR-s\nUpdated\n5 days ago\n•\n177k\n•\n1.17k\ngoogle/magenta-realtime\nUpdated\n3 days ago\n•\n316\nmistralai/Mistral-Small-3.2-24B-Instruct-2506\nUpdated\n4 days ago\n•\n5.37k\n•\n241\nMiniMaxAI/MiniMax-M1-80k\nUpdated\n1 day ago\n•\n10.4k\n•\n580\nOmniGen2/OmniGen2\nUpdated\n2 days ago\n•\n1.15k\n•\n160\nBrowse 1M+ models\nSpaces\nRunning\n828\n828\nSparc3D\n🏃\nNext-Gen High-Resolution 3D Model Generation\nRunning\n8.62k\n8.62k\nDeepSite v2\n🐳\nGenerate any application with DeepSeek\nRunning\non\nZero\n467\n467\nHunyuan3D-2.1\n👻\nImage-to-3D Generation\nRunning\non\nZero\n21